In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()  # Optional argument, if not specified will search path.

driver.get('https://www.redbus.in/');

#time.sleep(5) # Let the user actually see something!
driver.maximize_window()
driver.execute_script("window.scrollBy(0, window.innerHeight);")

view_all = driver.find_element(By.CSS_SELECTOR, 'a[href="https://www.redbus.in/online-booking/rtc-directory"]')
view_all.click();

#driver.quit()

In [2]:
# experimental code (kerala)
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
kerala = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/ksrtc-kerala"]')
kerala.click();
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

kerala_route_names = []
kerala_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        kerala_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        kerala_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(4)
    scrape_data()

# print(kerala_route_names)
# print(kerala_route_links)

df = pd.DataFrame(columns=['route_name', 'route_link', 'busname', 'bustype', 'departing_time', 
                           'duration', 'reaching_time', 'star_rating', 'price', 'seats_available'])
count = 0
for link in kerala_route_links:
    driver.get(link)
    driver.maximize_window()

    try:
        wait = WebDriverWait(driver, 3)
        wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        driver.execute_script("arguments[0].click();", wait_button)
        # government_buses = driver.find_element(By.CLASS_NAME, "button")
        #driver.execute_script("arguments[0].click();", government_buses)
        
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass    
    
    for t in range(22):     #scrolling 20 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 650);")
        time.sleep(0.3)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(kerala_route_names[count])
        route_link.append(link)

    count += 1


    # print(route_name)
    # print(route_link)
    # print(bus_name)
    # print(bus_type)
    # print(departing)
    # print(duration)
    # print(reaching)
    # print(rating)
    # print(price)
    # print(availability)

    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'


    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    #df.append(dict, ignore_index=True)
    df = pd.concat([df, df1], ignore_index=True)

    # con = mysql.connector.connect(
    # host="localhost",
    # user="root",
    # password="password"
    # )
    # cursor = con.cursor()

    # query = "create database if not exists REDBUS_DATA"
    # cursor.execute(query)

    # query = "use REDBUS_DATA"
    # cursor.execute(query)

    # query = """create table if not exists KERALA_KSRTC(id INT AUTO_INCREMENT PRIMARY KEY, route_name varchar(70), route_link varchar(180), busname varchar(100), bustype varchar(50), departing_time DATETIME,
    #     duration varchar(20), reaching_time DATETIME, star_rating float(2,1), price DECIMAL(6, 2), seats_available INT)"""
    # cursor.execute(query)
    
    # query = """insert into KERALA_KSRTC(route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available) 
    #         values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    # for i in range(len(bus_name)):
    #     data = (route_name[i],route_link[i],bus_name[i],bus_type[i],departing[i],duration[i],reaching[i],rating[i],price[i],availability[i])
    #     cursor.execute(query, data)
    #     con.commit()
print(df)    
    

                         route_name  \
0            Bangalore to Kozhikode   
1            Bangalore to Kozhikode   
2            Bangalore to Kozhikode   
3            Bangalore to Kozhikode   
4            Bangalore to Kozhikode   
..                              ...   
751  Kalpetta (kerala) to Kozhikode   
752  Kalpetta (kerala) to Kozhikode   
753  Kalpetta (kerala) to Kozhikode   
754  Kalpetta (kerala) to Kozhikode   
755  Kalpetta (kerala) to Kozhikode   

                                            route_link              busname  \
0    https://www.redbus.in/bus-tickets/bangalore-to...  Silver Line Travels   
1    https://www.redbus.in/bus-tickets/bangalore-to...          MMK Travels   
2    https://www.redbus.in/bus-tickets/bangalore-to...        Kyros Connect   
3    https://www.redbus.in/bus-tickets/bangalore-to...      AdDA Transports   
4    https://www.redbus.in/bus-tickets/bangalore-to...        NS Transports   
..                                                 ...   

In [3]:
df.to_csv('df_kerala.csv', index=False)

In [3]:
# experimental code (kadamba ktcl)
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
kadamba = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/ktcl"]')
kadamba.click();
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

kadamba_route_names = []
kadamba_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        kadamba_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        kadamba_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(2)
    scrape_data()

# print(kadamba_route_names)
# print(kadamba_route_links)


count = 0
for link in kadamba_route_links:
    driver.get(link)
    driver.maximize_window()
    

    for t in range(22):     #scrolling 16 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(0.5)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(kadamba_route_names[count])
        route_link.append(link)

    count += 1


    # print(route_name)
    # print(route_link)
    # print(bus_name)
    # print(bus_type)
    # print(departing)
    # print(duration)
    # print(reaching)
    # print(rating)
    # print(price)
    # print(availability)

    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'
    
    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)

    # con = mysql.connector.connect(
    # host="localhost",
    # user="root",
    # password="password"
    # )
    # cursor = con.cursor()

    # query = "create database if not exists REDBUS_DATA"
    # cursor.execute(query)

    # query = "use REDBUS_DATA"
    # cursor.execute(query)

    # query = """create table if not exists KADAMBA_KTCL(id INT AUTO_INCREMENT PRIMARY KEY, route_name varchar(70), route_link varchar(180), busname varchar(100), bustype varchar(50), departing_time DATETIME,
    #     duration varchar(20), reaching_time DATETIME, star_rating float(2,1), price DECIMAL(6, 2), seats_available INT)"""
    # cursor.execute(query)
    
    # query = """insert into KADAMBA_KTCL(route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available) 
    #         values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    # for i in range(len(bus_name)):
    #     data = (route_name[i],route_link[i],bus_name[i],bus_type[i],departing[i],duration[i],reaching[i],rating[i],price[i],availability[i])
    #     cursor.execute(query, data)
    #     con.commit()
print(df)   
    

                     route_name  \
0        Bangalore to Kozhikode   
1        Bangalore to Kozhikode   
2        Bangalore to Kozhikode   
3        Bangalore to Kozhikode   
4        Bangalore to Kozhikode   
...                         ...   
1196  Calangute (goa) to Panaji   
1197  Calangute (goa) to Panaji   
1198      Goa Airport to Panaji   
1199      Goa Airport to Panaji   
1200      Goa Airport to Panaji   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/bangalore-to...   
1     https://www.redbus.in/bus-tickets/bangalore-to...   
2     https://www.redbus.in/bus-tickets/bangalore-to...   
3     https://www.redbus.in/bus-tickets/bangalore-to...   
4     https://www.redbus.in/bus-tickets/bangalore-to...   
...                                                 ...   
1196  https://www.redbus.in/bus-tickets/calangute-go...   
1197  https://www.redbus.in/bus-tickets/calangute-go...   
1198  https://www.redbus.in/bus-tickets/goa-airp

In [4]:
# experimental code (Rajasthan RSRTC)
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
rajasthan = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/rsrtc"]')
driver.execute_script("arguments[0].click();", rajasthan) 
#driver.find_element(By.LINK_TEXT, 'RSRTC').click()
# rajasthan.click();
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

rajasthan_route_names = []
rajasthan_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        rajasthan_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        rajasthan_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(4)
    scrape_data()

# print(rajasthan_route_names)
# print(rajasthan_route_links)


count = 0
for link in rajasthan_route_links:
    driver.get(link)
    driver.maximize_window()

    try:
        wait = WebDriverWait(driver, 3)
        wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        driver.execute_script("arguments[0].click();", wait_button)
        # government_buses = driver.find_element(By.CLASS_NAME, "button")
        #driver.execute_script("arguments[0].click();", government_buses)
        
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass

    # government_buses = driver.find_element(By.CLASS_NAME, "button")
    # driver.execute_script("arguments[0].click();", government_buses)
    # time.sleep(1)

    for t in range(33):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 650);")
        time.sleep(0.25)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(rajasthan_route_names[count])
        route_link.append(link)

    count += 1


    # print(route_name)
    # print(route_link)
    # print(bus_name)
    # print(bus_type)
    # print(departing)
    # print(duration)
    # print(reaching)
    # print(rating)
    # print(price)
    # print(availability)

    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'

    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)

    # con = mysql.connector.connect(
    # host="localhost",
    # user="root",
    # password="password"
    # )
    # cursor = con.cursor()

    # query = "create database if not exists REDBUS_DATA"
    # cursor.execute(query)

    # query = "use REDBUS_DATA"
    # cursor.execute(query)

    # query = """create table if not exists RAJASTHAN_RSRTC(id INT AUTO_INCREMENT PRIMARY KEY, route_name varchar(70), route_link varchar(180), busname varchar(100), bustype varchar(50), departing_time DATETIME,
    #     duration varchar(20), reaching_time DATETIME, star_rating float(2,1), price DECIMAL(6, 2), seats_available INT)"""
    # cursor.execute(query)
    
    # query = """insert into RAJASTHAN_RSRTC(route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available) 
    #         values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    # for i in range(len(bus_name)):
    #     data = (route_name[i],route_link[i],bus_name[i],bus_type[i],departing[i],duration[i],reaching[i],rating[i],price[i],availability[i])
    #     cursor.execute(query, data)
    #     con.commit()
print(df)    
    

                  route_name  \
0     Bangalore to Kozhikode   
1     Bangalore to Kozhikode   
2     Bangalore to Kozhikode   
3     Bangalore to Kozhikode   
4     Bangalore to Kozhikode   
...                      ...   
2379        Bikaner to Sikar   
2380        Bikaner to Sikar   
2381        Bikaner to Sikar   
2382        Bikaner to Sikar   
2383        Bikaner to Sikar   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/bangalore-to...   
1     https://www.redbus.in/bus-tickets/bangalore-to...   
2     https://www.redbus.in/bus-tickets/bangalore-to...   
3     https://www.redbus.in/bus-tickets/bangalore-to...   
4     https://www.redbus.in/bus-tickets/bangalore-to...   
...                                                 ...   
2379  https://www.redbus.in/bus-tickets/bikaner-to-s...   
2380  https://www.redbus.in/bus-tickets/bikaner-to-s...   
2381  https://www.redbus.in/bus-tickets/bikaner-to-s...   
2382  https://www.redbus.

In [5]:
# experimental code (Himachal HRTC)
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
himachal = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/hrtc"]')
driver.execute_script("arguments[0].click();", himachal) 
time.sleep(2)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

himachal_route_names = []
himachal_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        himachal_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        himachal_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(1)
    scrape_data()

# print(himachal_route_names)
# print(himachal_route_links)


count = 0
for link in himachal_route_links:
    driver.get(link)
    driver.maximize_window()

    try:
        wait = WebDriverWait(driver, 3)
        wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        driver.execute_script("arguments[0].click();", wait_button)
        # government_buses = driver.find_element(By.CLASS_NAME, "button")
        #driver.execute_script("arguments[0].click();", government_buses)
        
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass

    # government_buses = driver.find_element(By.CLASS_NAME, "p-left-10")
    # driver.execute_script("arguments[0].click();", government_buses)
    # time.sleep(1)

    for t in range(33):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 650);")
        time.sleep(0.25)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(himachal_route_names[count])
        route_link.append(link)

    count += 1

    # print(route_name)
    # print(route_link)
    # print(bus_name)
    # print(bus_type)
    # print(departing)
    # print(duration)
    # print(reaching)
    # print(rating)
    # print(price)
    # print(availability)

    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'

    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)

    # con = mysql.connector.connect(
    # host="localhost",
    # user="root",
    # password="password"
    # )
    # cursor = con.cursor()

    # query = "create database if not exists REDBUS_DATA"
    # cursor.execute(query)

    # query = "use REDBUS_DATA"
    # cursor.execute(query)

    # query = """create table if not exists HIMACHAL_HRTC(id INT AUTO_INCREMENT PRIMARY KEY, route_name varchar(70), route_link varchar(180), busname varchar(100), bustype varchar(50), departing_time DATETIME,
    #     duration varchar(20), reaching_time DATETIME, star_rating float(2,1), price DECIMAL(6, 2), seats_available INT)"""
    # cursor.execute(query)
    
    # query = """insert into HIMACHAL_HRTC(route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available) 
    #         values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    # for i in range(len(bus_name)):
    #     data = (route_name[i],route_link[i],bus_name[i],bus_type[i],departing[i],duration[i],reaching[i],rating[i],price[i],availability[i])
    #     cursor.execute(query, data)
    #     con.commit()
print(df)
    

                  route_name  \
0     Bangalore to Kozhikode   
1     Bangalore to Kozhikode   
2     Bangalore to Kozhikode   
3     Bangalore to Kozhikode   
4     Bangalore to Kozhikode   
...                      ...   
3576      Delhi to Sarkaghat   
3577      Delhi to Sarkaghat   
3578      Delhi to Sarkaghat   
3579      Delhi to Sarkaghat   
3580      Delhi to Sarkaghat   

                                             route_link              busname  \
0     https://www.redbus.in/bus-tickets/bangalore-to...  Silver Line Travels   
1     https://www.redbus.in/bus-tickets/bangalore-to...          MMK Travels   
2     https://www.redbus.in/bus-tickets/bangalore-to...        Kyros Connect   
3     https://www.redbus.in/bus-tickets/bangalore-to...      AdDA Transports   
4     https://www.redbus.in/bus-tickets/bangalore-to...        NS Transports   
...                                                 ...                  ...   
3576  https://www.redbus.in/bus-tickets/delhi-to-sar...

In [6]:
# experimental code (punjab pepsu)
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
punjab = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/pepsu-punjab"]')
driver.execute_script("arguments[0].click();", punjab) 
time.sleep(2)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

punjab_route_names = []
punjab_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        punjab_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        punjab_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(1)
    scrape_data()

# print(punjab_route_names)
# print(punjab_route_links)


count = 0
for link in punjab_route_links:
    driver.get(link)
    driver.maximize_window()

    try:
        wait = WebDriverWait(driver, 3)
        wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        driver.execute_script("arguments[0].click();", wait_button)
        # government_buses = driver.find_element(By.CLASS_NAME, "button")
        #driver.execute_script("arguments[0].click();", government_buses)
        
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass

    # government_buses = driver.find_element(By.CLASS_NAME, "p-left-10")
    # driver.execute_script("arguments[0].click();", government_buses)
    # time.sleep(1)

    for t in range(32):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 650);")
        time.sleep(0.25)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(punjab_route_names[count])
        route_link.append(link)

    count += 1


    # print(route_name)
    # print(route_link)
    # print(bus_name)
    # print(bus_type)
    # print(departing)
    # print(duration)
    # print(reaching)
    # print(rating)
    # print(price)
    # print(availability)

    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'

    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)

    # con = mysql.connector.connect(
    # host="localhost",
    # user="root",
    # password="password"
    # )
    # cursor = con.cursor()

    # query = "create database if not exists REDBUS_DATA"
    # cursor.execute(query)

    # query = "use REDBUS_DATA"
    # cursor.execute(query)

    # query = """create table if not exists PUNJAB_PEPSU(id INT AUTO_INCREMENT PRIMARY KEY, route_name varchar(70), route_link varchar(180), busname varchar(100), bustype varchar(50), departing_time DATETIME,
    #     duration varchar(20), reaching_time DATETIME, star_rating float(2,1), price DECIMAL(6, 2), seats_available INT)"""
    # cursor.execute(query)
    
    # query = """insert into PUNJAB_PEPSU(route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available) 
    #         values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    # for i in range(len(bus_name)):
    #     data = (route_name[i],route_link[i],bus_name[i],bus_type[i],departing[i],duration[i],reaching[i],rating[i],price[i],availability[i])
    #     cursor.execute(query, data)
    #     con.commit()
print(df)    
    

                  route_name  \
0     Bangalore to Kozhikode   
1     Bangalore to Kozhikode   
2     Bangalore to Kozhikode   
3     Bangalore to Kozhikode   
4     Bangalore to Kozhikode   
...                      ...   
4189   Chandigarh to Patiala   
4190   Chandigarh to Patiala   
4191   Chandigarh to Patiala   
4192   Chandigarh to Patiala   
4193   Chandigarh to Patiala   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/bangalore-to...   
1     https://www.redbus.in/bus-tickets/bangalore-to...   
2     https://www.redbus.in/bus-tickets/bangalore-to...   
3     https://www.redbus.in/bus-tickets/bangalore-to...   
4     https://www.redbus.in/bus-tickets/bangalore-to...   
...                                                 ...   
4189  https://www.redbus.in/bus-tickets/chandigarh-t...   
4190  https://www.redbus.in/bus-tickets/chandigarh-t...   
4191  https://www.redbus.in/bus-tickets/chandigarh-t...   
4192  https://www.redbus.

In [7]:
# experimental code (BIHAR bsrtc)
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
bihar = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/bihar-state-road-transport-corporation-bsrtc"]')
driver.execute_script("arguments[0].click();", bihar) 
time.sleep(2)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

bihar_route_names = []
bihar_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        bihar_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        bihar_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(1)
    scrape_data()

# print(bihar_route_names)
# print(bihar_route_links)


count = 0
for link in bihar_route_links:
    driver.get(link)
    driver.maximize_window()

    try:
        wait = WebDriverWait(driver, 3)
        wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        driver.execute_script("arguments[0].click();", wait_button)
        # government_buses = driver.find_element(By.CLASS_NAME, "button")
        #driver.execute_script("arguments[0].click();", government_buses)
        
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass

    # government_buses = driver.find_element(By.CLASS_NAME, "p-left-10")
    # driver.execute_script("arguments[0].click();", government_buses)
    # time.sleep(1)

    for t in range(30):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 650);")
        time.sleep(0.25)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(bihar_route_names[count])
        route_link.append(link)

    count += 1


    # print(route_name)
    # print(route_link)
    # print(bus_name)
    # print(bus_type)
    # print(departing)
    # print(duration)
    # print(reaching)
    # print(rating)
    # print(price)
    # print(availability)

    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'

    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)

    # con = mysql.connector.connect(
    # host="localhost",
    # user="root",
    # password="password"
    # )
    # cursor = con.cursor()

    # query = "create database if not exists REDBUS_DATA"
    # cursor.execute(query)

    # query = "use REDBUS_DATA"
    # cursor.execute(query)

    # query = """create table if not exists BIHAR_BSRTC(id INT AUTO_INCREMENT PRIMARY KEY, route_name varchar(70), route_link varchar(180), busname varchar(100), bustype varchar(50), departing_time DATETIME,
    #     duration varchar(20), reaching_time DATETIME, star_rating float(2,1), price DECIMAL(6, 2), seats_available INT)"""
    # cursor.execute(query)
    
    # query = """insert into BIHAR_BSRTC(route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available) 
    #         values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    # for i in range(len(bus_name)):
    #     data = (route_name[i],route_link[i],bus_name[i],bus_type[i],departing[i],duration[i],reaching[i],rating[i],price[i],availability[i])
    #     cursor.execute(query, data)
    #     con.commit()
print(df)   
    

                        route_name  \
0           Bangalore to Kozhikode   
1           Bangalore to Kozhikode   
2           Bangalore to Kozhikode   
3           Bangalore to Kozhikode   
4           Bangalore to Kozhikode   
...                            ...   
4489  Lucknow to Gopalganj (Bihar)   
4490  Lucknow to Gopalganj (Bihar)   
4491  Lucknow to Gopalganj (Bihar)   
4492  Lucknow to Gopalganj (Bihar)   
4493  Lucknow to Gopalganj (Bihar)   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/bangalore-to...   
1     https://www.redbus.in/bus-tickets/bangalore-to...   
2     https://www.redbus.in/bus-tickets/bangalore-to...   
3     https://www.redbus.in/bus-tickets/bangalore-to...   
4     https://www.redbus.in/bus-tickets/bangalore-to...   
...                                                 ...   
4489  https://www.redbus.in/bus-tickets/lucknow-to-g...   
4490  https://www.redbus.in/bus-tickets/lucknow-to-g...   
4491  https:

In [8]:
# experimental code (south bengal sbstc)
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
s_bengal = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/south-bengal-state-transport-corporation-sbstc"]')
driver.execute_script("arguments[0].click();", s_bengal) 
time.sleep(2)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

s_bengal_route_names = []
s_bengal_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        s_bengal_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        s_bengal_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(1)
    scrape_data()

# print(s_bengal_route_names)
# print(s_bengal_route_links)


count = 0
for link in s_bengal_route_links:
    driver.get(link)
    driver.maximize_window()

    try:
        wait = WebDriverWait(driver, 3)
        wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        driver.execute_script("arguments[0].click();", wait_button)
        # government_buses = driver.find_element(By.CLASS_NAME, "button")
        #driver.execute_script("arguments[0].click();", government_buses)
        
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass

    # government_buses = driver.find_element(By.CLASS_NAME, "p-left-10")
    # driver.execute_script("arguments[0].click();", government_buses)
    # time.sleep(1)

    for t in range(25):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 650);")
        time.sleep(0.25)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict ={}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(s_bengal_route_names[count])
        route_link.append(link)

    count += 1


    # print(route_name)
    # print(route_link)
    # print(bus_name)
    # print(bus_type)
    # print(departing)
    # print(duration)
    # print(reaching)
    # print(rating)
    # print(price)
    # print(availability)

    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'

    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)

    # con = mysql.connector.connect(
    # host="localhost",
    # user="root",
    # password="password"
    # )
    # cursor = con.cursor()

    # query = "create database if not exists REDBUS_DATA"
    # cursor.execute(query)

    # query = "use REDBUS_DATA"
    # cursor.execute(query)

    # query = """create table if not exists S_BENGAL_SBRTC(id INT AUTO_INCREMENT PRIMARY KEY, route_name varchar(70), route_link varchar(180), busname varchar(100), bustype varchar(50), departing_time DATETIME,
    #     duration varchar(20), reaching_time DATETIME, star_rating float(2,1), price DECIMAL(6, 2), seats_available INT)"""
    # cursor.execute(query)
    
    # query = """insert into S_BENGAL_SBRTC(route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available) 
    #         values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    # for i in range(len(bus_name)):
    #     data = (route_name[i],route_link[i],bus_name[i],bus_type[i],departing[i],duration[i],reaching[i],rating[i],price[i],availability[i])
    #     cursor.execute(query, data)
    #     con.commit()
print(df)    
    

                  route_name  \
0     Bangalore to Kozhikode   
1     Bangalore to Kozhikode   
2     Bangalore to Kozhikode   
3     Bangalore to Kozhikode   
4     Bangalore to Kozhikode   
...                      ...   
5341         Kolkata to Suri   
5342         Kolkata to Suri   
5343         Kolkata to Suri   
5344         Kolkata to Suri   
5345         Kolkata to Suri   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/bangalore-to...   
1     https://www.redbus.in/bus-tickets/bangalore-to...   
2     https://www.redbus.in/bus-tickets/bangalore-to...   
3     https://www.redbus.in/bus-tickets/bangalore-to...   
4     https://www.redbus.in/bus-tickets/bangalore-to...   
...                                                 ...   
5341  https://www.redbus.in/bus-tickets/kolkata-to-suri   
5342  https://www.redbus.in/bus-tickets/kolkata-to-suri   
5343  https://www.redbus.in/bus-tickets/kolkata-to-suri   
5344  https://www.redbus.

In [9]:
# experimental code (West bengal)
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
w_bengal = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/west-bengal-transport-corporation"]')
driver.execute_script("arguments[0].click();", w_bengal) 
time.sleep(2)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

w_bengal_route_names = []
w_bengal_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        w_bengal_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        w_bengal_route_links.append(href)

# page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
# for i in range(len(page_tabs)):
#     if i > 0:
#         page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
#         driver.execute_script("arguments[0].click();", page_tabs)        
#         time.sleep(1)
#     scrape_data()

scrape_data()

# print(w_bengal_route_names)
# print(w_bengal_route_links)


count = 0
for link in w_bengal_route_links:
    driver.get(link)
    driver.maximize_window()

    # government_buses = driver.find_element(By.CLASS_NAME, "p-left-10")
    # driver.execute_script("arguments[0].click();", government_buses)
    # time.sleep(1)

    for t in range(20):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(0.3)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(w_bengal_route_names[count])
        route_link.append(link)

    count += 1

    # print(route_name)
    # print(route_link)
    # print(bus_name)
    # print(bus_type)
    # print(departing)
    # print(duration)
    # print(reaching)
    # print(rating)
    # print(price)
    # print(availability)

    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'

    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)

    # con = mysql.connector.connect(
    # host="localhost",
    # user="root",
    # password="password"
    # )
    # cursor = con.cursor()

    # query = "create database if not exists REDBUS_DATA"
    # cursor.execute(query)

    # query = "use REDBUS_DATA"
    # cursor.execute(query)

    # query = """create table if not exists WEST_BENGAL(id INT AUTO_INCREMENT PRIMARY KEY, route_name varchar(70), route_link varchar(180), busname varchar(100), bustype varchar(50), departing_time DATETIME,
    #     duration varchar(20), reaching_time DATETIME, star_rating float(2,1), price DECIMAL(6, 2), seats_available INT)"""
    # cursor.execute(query)
    
    # query = """insert into WEST_BENGAL(route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available) 
    #         values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    # for i in range(len(bus_name)):
    #     data = (route_name[i],route_link[i],bus_name[i],bus_type[i],departing[i],duration[i],reaching[i],rating[i],price[i],availability[i])
    #     cursor.execute(query, data)
    #     con.commit()
print(df)    
    

                  route_name  \
0     Bangalore to Kozhikode   
1     Bangalore to Kozhikode   
2     Bangalore to Kozhikode   
3     Bangalore to Kozhikode   
4     Bangalore to Kozhikode   
...                      ...   
5546   Kolkata to Mandarmani   
5547   Kolkata to Mandarmani   
5548   Kolkata to Mandarmani   
5549   Kolkata to Mandarmani   
5550   Kolkata to Mandarmani   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/bangalore-to...   
1     https://www.redbus.in/bus-tickets/bangalore-to...   
2     https://www.redbus.in/bus-tickets/bangalore-to...   
3     https://www.redbus.in/bus-tickets/bangalore-to...   
4     https://www.redbus.in/bus-tickets/bangalore-to...   
...                                                 ...   
5546  https://www.redbus.in/bus-tickets/kolkata-to-m...   
5547  https://www.redbus.in/bus-tickets/kolkata-to-m...   
5548  https://www.redbus.in/bus-tickets/kolkata-to-m...   
5549  https://www.redbus.

In [10]:
# experimental code (Chandigarh CTU)
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
chandigarh = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/chandigarh-transport-undertaking-ctu"]')
driver.execute_script("arguments[0].click();", chandigarh) 
time.sleep(2)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

chandigarh_route_names = []
chandigarh_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        chandigarh_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        chandigarh_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(1)
    scrape_data()


# print(chandigarh_route_names)
# print(chandigarh_route_links)


count = 0
for link in chandigarh_route_links:
    driver.get(link)
    driver.maximize_window()

    try:
        wait = WebDriverWait(driver, 3)
        wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        driver.execute_script("arguments[0].click();", wait_button)
        # government_buses = driver.find_element(By.CLASS_NAME, "button")
        #driver.execute_script("arguments[0].click();", government_buses)
        
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass

    # government_buses = driver.find_element(By.CLASS_NAME, "p-left-10")
    # driver.execute_script("arguments[0].click();", government_buses)
    # time.sleep(1)

    for t in range(20):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(0.3)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(chandigarh_route_names[count])
        route_link.append(link)

    count += 1


    # print(route_name)
    # print(route_link)
    # print(bus_name)
    # print(bus_type)
    # print(departing)
    # print(duration)
    # print(reaching)
    # print(rating)
    # print(price)
    # print(availability)

    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'

    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)

    # con = mysql.connector.connect(
    # host="localhost",
    # user="root",
    # password="password"
    # )
    # cursor = con.cursor()

    # query = "create database if not exists REDBUS_DATA"
    # cursor.execute(query)

    # query = "use REDBUS_DATA"
    # cursor.execute(query)

    # query = """create table if not exists CHANDIGARH_CTU(id INT AUTO_INCREMENT PRIMARY KEY, route_name varchar(70), route_link varchar(180), busname varchar(100), bustype varchar(50), departing_time DATETIME,
    #     duration varchar(20), reaching_time DATETIME, star_rating float(2,1), price DECIMAL(6, 2), seats_available INT)"""
    # cursor.execute(query)
    
    # query = """insert into CHANDIGARH_CTU(route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available) 
    #         values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    # for i in range(len(bus_name)):
    #     data = (route_name[i],route_link[i],bus_name[i],bus_type[i],departing[i],duration[i],reaching[i],rating[i],price[i],availability[i])
    #     cursor.execute(query, data)
    #     con.commit()
print(df)    
    

                   route_name  \
0      Bangalore to Kozhikode   
1      Bangalore to Kozhikode   
2      Bangalore to Kozhikode   
3      Bangalore to Kozhikode   
4      Bangalore to Kozhikode   
...                       ...   
6393    Narnaul to Chandigarh   
6394    Narnaul to Chandigarh   
6395    Narnaul to Chandigarh   
6396  Chandigarh to Jawala Ji   
6397  Chandigarh to Jawala Ji   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/bangalore-to...   
1     https://www.redbus.in/bus-tickets/bangalore-to...   
2     https://www.redbus.in/bus-tickets/bangalore-to...   
3     https://www.redbus.in/bus-tickets/bangalore-to...   
4     https://www.redbus.in/bus-tickets/bangalore-to...   
...                                                 ...   
6393  https://www.redbus.in/bus-tickets/narnaul-to-c...   
6394  https://www.redbus.in/bus-tickets/narnaul-to-c...   
6395  https://www.redbus.in/bus-tickets/narnaul-to-c...   
6396  https:/

In [11]:
# experimental code (Assam ASTC)
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
assam = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/astc"]')
driver.execute_script("arguments[0].click();", assam) 
time.sleep(2)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

assam_route_names = []
assam_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        assam_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        assam_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(1)
    scrape_data()


# print(assam_route_names)
# print(assam_route_links)


count = 0
for link in assam_route_links:
    driver.get(link)
    driver.maximize_window()

    try:
        wait = WebDriverWait(driver, 3)
        wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        driver.execute_script("arguments[0].click();", wait_button)
        # government_buses = driver.find_element(By.CLASS_NAME, "button")
        #driver.execute_script("arguments[0].click();", government_buses)
        
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass

    # government_buses = driver.find_element(By.CLASS_NAME, "p-left-10")
    # driver.execute_script("arguments[0].click();", government_buses)
    # time.sleep(1)

    for t in range(20):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(0.3)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(assam_route_names[count])
        route_link.append(link)

    count += 1


    # print(route_name)
    # print(route_link)
    # print(bus_name)
    # print(bus_type)
    # print(departing)
    # print(duration)
    # print(reaching)
    # print(rating)
    # print(price)
    # print(availability)

    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'

    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)

    # con = mysql.connector.connect(
    # host="localhost",
    # user="root",
    # password="password"
    # )
    # cursor = con.cursor()

    # query = "create database if not exists REDBUS_DATA"
    # cursor.execute(query)

    # query = "use REDBUS_DATA"
    # cursor.execute(query)

    # query = """create table if not exists ASSAM_ASTC(id INT AUTO_INCREMENT PRIMARY KEY, route_name varchar(70), route_link varchar(180), busname varchar(100), bustype varchar(50), departing_time DATETIME,
    #     duration varchar(20), reaching_time DATETIME, star_rating float(2,1), price DECIMAL(6, 2), seats_available INT)"""
    # cursor.execute(query)
    
    # query = """insert into ASSAM_ASTC(route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available) 
    #         values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    # for i in range(len(bus_name)):
    #     data = (route_name[i],route_link[i],bus_name[i],bus_type[i],departing[i],duration[i],reaching[i],rating[i],price[i],availability[i])
    #     cursor.execute(query, data)
    #     con.commit()
print(df)
    

                  route_name  \
0     Bangalore to Kozhikode   
1     Bangalore to Kozhikode   
2     Bangalore to Kozhikode   
3     Bangalore to Kozhikode   
4     Bangalore to Kozhikode   
...                      ...   
6645      Tezpur to Tinsukia   
6646   Dibrugarh to Bokakhat   
6647   Dibrugarh to Bokakhat   
6648   Dibrugarh to Bokakhat   
6649   Dibrugarh to Bokakhat   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/bangalore-to...   
1     https://www.redbus.in/bus-tickets/bangalore-to...   
2     https://www.redbus.in/bus-tickets/bangalore-to...   
3     https://www.redbus.in/bus-tickets/bangalore-to...   
4     https://www.redbus.in/bus-tickets/bangalore-to...   
...                                                 ...   
6645  https://www.redbus.in/bus-tickets/tezpur-to-ti...   
6646  https://www.redbus.in/bus-tickets/dibrugarh-to...   
6647  https://www.redbus.in/bus-tickets/dibrugarh-to...   
6648  https://www.redbus.

In [12]:
df.to_csv('final_data1.csv')

In [10]:
df = pd.read_csv('final_data.csv')
df = df.reset_index(drop=True)
print(df.dtypes)
df = df.astype({'seats_available': 'int'})
print(df.dtypes)

route_name          object
route_link          object
busname             object
bustype             object
departing_time      object
duration            object
reaching_time       object
star_rating        float64
price              float64
seats_available      int64
dtype: object
route_name          object
route_link          object
busname             object
bustype             object
departing_time      object
duration            object
reaching_time       object
star_rating        float64
price              float64
seats_available      int64
dtype: object


In [12]:
df

,route_name,route_link,busname,bustype,departing_time,duration,reaching_time,star_rating,price,seats_available
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1210,SWIFT-HYBRID NON AC SEATER CUM SLEEPER,12:55,08h 20m,21:15,3.8,567.0,7
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1378,AC MULTI AXLE,13:46,07h 04m,20:50,2.7,729.0,27
2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,14:15,06h 44m,20:59,4.0,627.0,22
3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,09h 01m,00:01,4.6,620.0,18
4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 370,AC MULTI AXLE,15:25,06h 55m,22:20,2.3,729.0,19
...,...,...,...,...,...,...,...,...,...,...
5362,Dibrugarh to Bokakhat,https://www.redbus.in/bus-tickets/dibrugarh-to...,Baikuntha Transport Service,Bharat Benz A/C Seater /Sleeper (2+1),19:30,05h 30m,01:00,3.9,540.0,27
5363,Dibrugarh to Bokakhat,https://www.redbus.in/bus-tickets/dibrugarh-to...,Purple Wings Coaches,A/C Seater / Sleeper (2+1),19:55,05h 05m,01:00,3.4,540.0,32
5364,Dibrugarh to Bokakhat,https://www.redbus.in/bus-tickets/dibrugarh-to...,Shree Ram Travels (Under ASTC),Bharat Benz A/C Seater (2+1),20:00,05h 00m,01:00,4.8,500.0,18
5365,Dibrugarh to Bokakhat,https://www.redbus.in/bus-tickets/dibrugarh-to...,Shree Ram Travels,Bharat Benz A/C Seater /Sleeper (2+1),20:45,04h 55m,01:40,4.8,530.0,16


In [13]:
import pandas as pd
import mysql.connector
con = mysql.connector.connect(
host="localhost",
user="root",
password="password"
)
cursor = con.cursor()

query = "create database if not exists REDBUS_DATA_SCRAPING_PROJECT"
cursor.execute(query)

query = "use REDBUS_DATA_SCRAPING_PROJECT"
cursor.execute(query)

query = """create table if not exists All_bus_data(id INT AUTO_INCREMENT PRIMARY KEY, route_name varchar(70), 
        route_link varchar(180), busname varchar(100), bustype varchar(50), departing_time TIME,
        duration varchar(20), reaching_time TIME, star_rating float(2,1), price DECIMAL(7, 2), seats_available INT)"""
cursor.execute(query)

query = """insert into All_bus_data(route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available) 
        values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
for i in range(len(df)):
    data = (df.loc[i, 'route_name'], df.loc[i, 'route_link'], df.loc[i, 'busname'], df.loc[i, 'bustype'], df.loc[i, 'departing_time'], df.loc[i, 'duration'], 
            df.loc[i, 'reaching_time'], df.loc[i, 'star_rating'], df.loc[i, 'price'], df.loc[i, 'seats_available'])
    cursor.execute(query, data)
    con.commit()

In [10]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
kerala = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/ksrtc-kerala"]')
kerala.click();
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")


rajasthan_route_names = []
rajasthan_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        rajasthan_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        rajasthan_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(4)
    scrape_data()

print(rajasthan_route_names)
print(rajasthan_route_links)

['Bangalore to Kozhikode', 'Kozhikode to Ernakulam', 'Kozhikode to Bangalore', 'Ernakulam to Kozhikode', 'Kozhikode to Mysore', 'Kozhikode to Thiruvananthapuram', 'Bangalore to Kalpetta (kerala)', 'Mysore to Kozhikode', 'Kalpetta (kerala) to Bangalore', 'Kozhikode to Thrissur', 'Thiruvananthapuram to Kozhikode', 'Bangalore to Kannur', 'Kozhikode to Kottayam', 'Kannur to Bangalore', 'Kottayam to Kozhikode', 'Thrissur to Kozhikode', 'Kozhikode to Kalpetta (kerala)', 'Coimbatore to Ooty', 'Kalpetta (kerala) to Kozhikode']
['https://www.redbus.in/bus-tickets/bangalore-to-kozhikode', 'https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam', 'https://www.redbus.in/bus-tickets/kozhikode-to-bangalore', 'https://www.redbus.in/bus-tickets/ernakulam-to-kozhikode', 'https://www.redbus.in/bus-tickets/kozhikode-to-mysore', 'https://www.redbus.in/bus-tickets/kozhikode-to-thiruvananthapuram', 'https://www.redbus.in/bus-tickets/bangalore-to-kalpetta', 'https://www.redbus.in/bus-tickets/mysore-to-kozh

In [4]:
import mysql.connector
con = mysql.connector.connect(
    host="localhost",
    user="root",
    password="password"
)
cursor = con.cursor()

query = "create database if not exists REDBUS_DATA"
cursor.execute(query)

query = "use REDBUS_DATA"
cursor.execute(query)

query = """create table if not exists KERALA_KSRTC(route_name varchar(50), Route_Address varchar(150), Bus_Names varchar(40), bus_type varchar(50), Time_of_Departure varchar(10),
        Travel_Duration varchar(10), Time_of_Arrival varchar(10), Star_Ratings float(4), Fare varchar(10), Seats_availability varchar(50))"""
cursor.execute(query)



In [56]:
for link in rajasthan_route_links:
    print(link)
    driver.get(link)
    driver.maximize_window()
    time.sleep(3)
    
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"lh-18")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text)

    for fare in fares:
        price.append(fare.text)

    for seat_available in seats_available:
        availability.append(seat_available.text)
    break
print(bus_name)
print(bus_type)
print(departing)
print(duration)
print(reaching)
print(rating)
print(price)
print(availability)



https://www.redbus.in/bus-tickets/bangalore-to-kozhikode


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF6845DEEA2+31554]
	(No symbol) [0x00007FF684557ED9]
	(No symbol) [0x00007FF68441872A]
	(No symbol) [0x00007FF6843FFA9C]
	(No symbol) [0x00007FF6843FF960]
	(No symbol) [0x00007FF68441AD51]
	(No symbol) [0x00007FF6844AC079]
	(No symbol) [0x00007FF68448CDD3]
	(No symbol) [0x00007FF68445A33B]
	(No symbol) [0x00007FF68445AED1]
	GetHandleVerifier [0x00007FF6848E8B1D+3217341]
	GetHandleVerifier [0x00007FF684935AE3+3532675]
	GetHandleVerifier [0x00007FF68492B0E0+3489152]
	GetHandleVerifier [0x00007FF68468E776+750614]
	(No symbol) [0x00007FF68456375F]
	(No symbol) [0x00007FF68455EB14]
	(No symbol) [0x00007FF68455ECA2]
	(No symbol) [0x00007FF68454E16F]
	BaseThreadInitThunk [0x00007FFCCF22257D+29]
	RtlUserThreadStart [0x00007FFCCFE4AF28+40]


In [9]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()  # Optional argument, if not specified will search path.

driver.get('https://www.redbus.in/');

#time.sleep(5) # Let the user actually see something!
driver.maximize_window()
driver.execute_script("window.scrollBy(0, window.innerHeight);")

view_all = driver.find_element(By.CSS_SELECTOR, 'a[href="https://www.redbus.in/online-booking/rtc-directory"]')
view_all.click();
driver.maximize_window()
#driver.quit()

In [18]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
kerala = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/ksrtc-kerala"]')
kerala.click();
time.sleep(4)
driver.execute_script("window.scrollBy(0, window.innerHeight);")

kerala_route_names_prices = []
rajasthan_route_names = []
rajasthan_route_links = []
count = 1

page_tabs = driver.find_elements(By.CLASS_NAME,"DC_117_pageTabs ")
#page_tabs_nums = len(page_tabs)
#print(page_tabs_nums)

#for page_tab in page_tabs:
#    print(page_tab.text)


for i in range(len(page_tabs)):
    # Re-find the page tabs to avoid stale element reference
    page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")

    if count > 1:
        page_tabs[i].click()

    #for i in range(page_tabs_nums):
    #    driver.find_elements(By.CLASS_NAME,"DC_117_pageTabs ").text

    route_names = driver.find_elements(By.CLASS_NAME,"route_details")

    for route_name in route_names:
        kerala_route_names_prices.append(route_name.text)
        #print(route_name.text)

    for kerala_route_name_price in kerala_route_names_prices:
        rajasthan_route_names.append(kerala_route_name_price.split('\n')[0])

    route_links = driver.find_elements(By.CLASS_NAME,"route")

    hrefs = [route_link.get_attribute('href') for route_link in route_links]

    for href in hrefs:
        rajasthan_route_links.append(href)

    count += 1

print(rajasthan_route_names)
print(rajasthan_route_links)



ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (694, 1470)
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF6845DEEA2+31554]
	(No symbol) [0x00007FF684557ED9]
	(No symbol) [0x00007FF68441872A]
	(No symbol) [0x00007FF68447012E]
	(No symbol) [0x00007FF68446DAF2]
	(No symbol) [0x00007FF68446AF8B]
	(No symbol) [0x00007FF68446A156]
	(No symbol) [0x00007FF68445C151]
	(No symbol) [0x00007FF68448D02A]
	(No symbol) [0x00007FF68445BA76]
	(No symbol) [0x00007FF68448D240]
	(No symbol) [0x00007FF6844AC977]
	(No symbol) [0x00007FF68448CDD3]
	(No symbol) [0x00007FF68445A33B]
	(No symbol) [0x00007FF68445AED1]
	GetHandleVerifier [0x00007FF6848E8B1D+3217341]
	GetHandleVerifier [0x00007FF684935AE3+3532675]
	GetHandleVerifier [0x00007FF68492B0E0+3489152]
	GetHandleVerifier [0x00007FF68468E776+750614]
	(No symbol) [0x00007FF68456375F]
	(No symbol) [0x00007FF68455EB14]
	(No symbol) [0x00007FF68455ECA2]
	(No symbol) [0x0

In [23]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
kerala = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/ksrtc-kerala"]')
kerala.click();
time.sleep(4)
driver.execute_script("window.scrollBy(0, window.innerHeight);")

kerala_route_names_prices = []
rajasthan_route_names = []
rajasthan_route_links = []


page_tabs = driver.find_elements(By.CLASS_NAME,"DC_117_pageTabs ")
#page_tabs_nums = len(page_tabs)
#print(page_tabs_nums)

#for page_tab in page_tabs:
#    print(page_tab.text)


for i in range(len(page_tabs)):


    #for i in range(page_tabs_nums):
    #    driver.find_elements(By.CLASS_NAME,"DC_117_pageTabs ").text

    route_names = driver.find_elements(By.CLASS_NAME,"route_details")

    for route_name in route_names:
        kerala_route_names_prices.append(route_name.text)
        #print(route_name.text)

    for kerala_route_name_price in kerala_route_names_prices:
        rajasthan_route_names.append(kerala_route_name_price.split('\n')[0])

    route_links = driver.find_elements(By.CLASS_NAME,"route")

    hrefs = [route_link.get_attribute('href') for route_link in route_links]

    for href in hrefs:
        rajasthan_route_links.append(href)

    print(rajasthan_route_names)
    print(rajasthan_route_links)

        # Re-find the page tabs to avoid stale element reference
    page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
    #print(page_tabs)

    if i < (len(page_tabs)-1):
        page_tabs[i+1].click()
    
    

print(rajasthan_route_names)
print(rajasthan_route_links)



['Bangalore to Kozhikode', 'Kozhikode to Ernakulam', 'Kozhikode to Bangalore', 'Ernakulam to Kozhikode', 'Kozhikode to Mysore', 'Kozhikode to Thiruvananthapuram', 'Bangalore to Kalpetta (kerala)', 'Mysore to Kozhikode', 'Kalpetta (kerala) to Bangalore', 'Kozhikode to Thrissur']
['https://www.redbus.in/bus-tickets/bangalore-to-kozhikode', 'https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam', 'https://www.redbus.in/bus-tickets/kozhikode-to-bangalore', 'https://www.redbus.in/bus-tickets/ernakulam-to-kozhikode', 'https://www.redbus.in/bus-tickets/kozhikode-to-mysore', 'https://www.redbus.in/bus-tickets/kozhikode-to-thiruvananthapuram', 'https://www.redbus.in/bus-tickets/bangalore-to-kalpetta', 'https://www.redbus.in/bus-tickets/mysore-to-kozhikode', 'https://www.redbus.in/bus-tickets/kalpetta-to-bangalore', 'https://www.redbus.in/bus-tickets/kozhikode-to-thrissur']


ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (694, 664)
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF6845DEEA2+31554]
	(No symbol) [0x00007FF684557ED9]
	(No symbol) [0x00007FF68441872A]
	(No symbol) [0x00007FF68447012E]
	(No symbol) [0x00007FF68446DAF2]
	(No symbol) [0x00007FF68446AF8B]
	(No symbol) [0x00007FF68446A156]
	(No symbol) [0x00007FF68445C151]
	(No symbol) [0x00007FF68448D02A]
	(No symbol) [0x00007FF68445BA76]
	(No symbol) [0x00007FF68448D240]
	(No symbol) [0x00007FF6844AC977]
	(No symbol) [0x00007FF68448CDD3]
	(No symbol) [0x00007FF68445A33B]
	(No symbol) [0x00007FF68445AED1]
	GetHandleVerifier [0x00007FF6848E8B1D+3217341]
	GetHandleVerifier [0x00007FF684935AE3+3532675]
	GetHandleVerifier [0x00007FF68492B0E0+3489152]
	GetHandleVerifier [0x00007FF68468E776+750614]
	(No symbol) [0x00007FF68456375F]
	(No symbol) [0x00007FF68455EB14]
	(No symbol) [0x00007FF68455ECA2]
	(No symbol) [0x00